<a href="https://colab.research.google.com/github/wtrekell/syntaxandempathy/blob/main/30-articles/ai-vs-human/Colab%20Notebooks/ai-vs-human-v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# CELL 1: SETUP AND CONFIGURATION
# =============================================================================

import os
import re
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
from google.colab import drive, files
import zipfile

# Mount Google Drive (will prompt for authorization)
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("✓ Drive mounted successfully")

# Configuration
class Config:
    """Configuration settings for the analysis"""

    # File naming conventions
    VERSION_PREFIXES = ['draft-', 'refined-', 'edited-', 'final-']
    VERSION_ORDER = {prefix: i for i, prefix in enumerate(VERSION_PREFIXES)}

    # Analysis settings
    MIN_SENTENCE_LENGTH = 10  # Minimum characters for a sentence
    MAX_SENTENCE_LENGTH = 1000  # Maximum characters for a sentence

def setup_output_directories(base_path):
    """Create necessary output directories"""
    output_dir = os.path.join(base_path, 'output')
    archive_dir = os.path.join(base_path, 'archive')

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(archive_dir, exist_ok=True)

    print(f"✓ Output directory ready: {output_dir}")
    print(f"✓ Archive directory ready: {archive_dir}")

    return output_dir, archive_dir

print("📋 Configuration loaded. Ready to process articles.")

Mounting Google Drive...
Mounted at /content/drive
✓ Drive mounted successfully
📋 Configuration loaded. Ready to process articles.


In [ ]:
# =============================================================================
# CELL 2: DATA INGESTION & VALIDATION (STEP 1)
# =============================================================================

class ArticleVersions:
    """Class to handle loading and validating article versions"""

    def __init__(self, article_name, input_path):
        self.article_name = article_name
        self.input_path = input_path
        self.versions = {}
        self.metadata = {
            'article_name': article_name,
            'input_path': input_path,
            'processing_timestamp': datetime.now().isoformat(),
            'versions_found': [],
            'validation_status': 'pending'
        }

    def load_versions(self):
        """Load all versions of an article from the specified path"""
        print(f"\n📁 Loading versions for article: {self.article_name}")
        print(f"📂 Input path: {self.input_path}")

        # Find all files matching the article name pattern
        for prefix in Config.VERSION_PREFIXES:
            filename = f"{prefix}{self.article_name}.md"
            filepath = os.path.join(self.input_path, filename)

            if os.path.exists(filepath):
                try:
                    with open(filepath, 'r', encoding='utf-8') as file:
                        content = file.read()
                        self.versions[prefix.rstrip('-')] = {
                            'filename': filename,
                            'filepath': filepath,
                            'content': content,
                            'loaded_at': datetime.now().isoformat(),
                            'file_size': len(content)
                        }
                        print(f"  ✓ Loaded: {filename} ({len(content)} characters)")

                except Exception as e:
                    print(f"  ✗ Error loading {filename}: {str(e)}")
            else:
                print(f"  - Not found: {filename}")

        self.metadata['versions_found'] = list(self.versions.keys())
        return self.versions

    def validate_version_sequence(self):
        """Validate that we have the minimum required versions"""
        found_versions = set(self.versions.keys())
        required_versions = ['draft', 'final']

        # Check for required versions
        missing_required = []
        for version in required_versions:
            if version not in found_versions:
                missing_required.append(version)

        # Validation results
        validation_results = {
            'has_draft': 'draft' in found_versions,
            'has_final': 'final' in found_versions,
            'missing_required': missing_required,
            'versions_found': list(found_versions),
            'is_valid': len(missing_required) == 0
        }

        # Update metadata
        self.metadata['validation_results'] = validation_results

        if validation_results['is_valid']:
            self.metadata['validation_status'] = 'passed'
            print(f"✓ Validation passed: Found {len(found_versions)} versions")
        else:
            self.metadata['validation_status'] = 'failed'
            print(f"✗ Validation failed: Missing required versions")
            print(f"  Missing: {', '.join(missing_required)}")

        return validation_results

    def get_summary(self):
        """Get a summary of loaded versions"""
        summary = {
            'article_name': self.article_name,
            'input_path': self.input_path,
            'versions_count': len(self.versions),
            'validation_status': self.metadata['validation_status'],
            'file_sizes': {}
        }

        for version, data in self.versions.items():
            summary['file_sizes'][version] = data['file_size']

        return summary

print("📖 ArticleVersions class loaded. Ready for data ingestion.")

📖 ArticleVersions class loaded. Ready for data ingestion.


In [ ]:
# =============================================================================
# CELL 3: TEXT PREPROCESSING (STEP 2)
# =============================================================================

class TextPreprocessor:
    """Class to handle text preprocessing and segmentation"""

    def __init__(self):
        self.processed_versions = {}

    def clean_markdown(self, text):
        """Clean markdown formatting while preserving content structure"""
        # Remove markdown formatting but keep the text
        patterns = [
            (r'^\s*#{1,6}\s+', ''),  # Headers
            (r'\*\*(.*?)\*\*', r'\1'),  # Bold
            (r'\*(.*?)\*', r'\1'),  # Italic
            (r'`(.*?)`', r'\1'),  # Inline code
            (r'```.*?```', ''),  # Code blocks
            (r'!\[.*?\]\(.*?\)', ''),  # Images
            (r'\[([^\]]+)\]\([^\)]+\)', r'\1'),  # Links
            (r'^\s*[\*\-\+]\s+', '', re.MULTILINE),  # Bullet points
            (r'^\s*\d+\.\s+', '', re.MULTILINE),  # Numbered lists
            (r'\n{3,}', '\n\n'),  # Multiple newlines
        ]

        cleaned_text = text
        for pattern, replacement in patterns[:-1]:  # All except the last one
            cleaned_text = re.sub(pattern, replacement, cleaned_text)

        # Handle the multiline pattern separately
        cleaned_text = re.sub(patterns[-1][0], patterns[-1][1], cleaned_text, flags=re.MULTILINE)

        return cleaned_text.strip()

    def segment_into_sentences(self, text):
        """Segment text into sentences with basic filtering"""
        # Simple sentence segmentation (can be enhanced with spaCy later if needed)
        sentences = re.split(r'[.!?]+\s+', text)

        # Filter sentences
        filtered_sentences = []
        for sentence in sentences:
            sentence = sentence.strip()
            if (Config.MIN_SENTENCE_LENGTH <= len(sentence) <= Config.MAX_SENTENCE_LENGTH
                and sentence):
                filtered_sentences.append(sentence)

        return filtered_sentences

    def segment_into_paragraphs(self, text):
        """Segment text into paragraphs"""
        paragraphs = [p.strip() for p in text.split('\n\n') if p.strip()]
        return paragraphs

    def process_version(self, version_name, raw_content):
        """Process a single version of the article"""
        print(f"  Processing {version_name} version...")

        # Clean the markdown
        cleaned_content = self.clean_markdown(raw_content)

        # Segment into different units
        sentences = self.segment_into_sentences(cleaned_content)
        paragraphs = self.segment_into_paragraphs(cleaned_content)

        # Calculate basic statistics
        stats = {
            'character_count': len(cleaned_content),
            'word_count': len(cleaned_content.split()),
            'sentence_count': len(sentences),
            'paragraph_count': len(paragraphs),
            'avg_sentence_length': sum(len(s) for s in sentences) / len(sentences) if sentences else 0,
            'avg_paragraph_length': sum(len(p) for p in paragraphs) / len(paragraphs) if paragraphs else 0
        }

        processed_data = {
            'version_name': version_name,
            'raw_content': raw_content,
            'cleaned_content': cleaned_content,
            'sentences': sentences,
            'paragraphs': paragraphs,
            'statistics': stats,
            'processed_at': datetime.now().isoformat()
        }

        self.processed_versions[version_name] = processed_data

        print(f"    ✓ {stats['sentence_count']} sentences, {stats['paragraph_count']} paragraphs")
        print(f"    ✓ {stats['word_count']} words, {stats['character_count']} characters")

        return processed_data

    def process_all_versions(self, article_versions):
        """Process all versions of an article"""
        print(f"\n🔄 Preprocessing text for all versions...")

        for version_name, version_data in article_versions.versions.items():
            self.process_version(version_name, version_data['content'])

        return self.processed_versions

    def get_processing_summary(self):
        """Get a summary of processing results"""
        summary = {}
        for version_name, data in self.processed_versions.items():
            summary[version_name] = data['statistics']

        return summary

print("🔧 TextPreprocessor class loaded. Ready for text processing.")

🔧 TextPreprocessor class loaded. Ready for text processing.


In [ ]:
# =============================================================================
# CELL 4: EXECUTION FUNCTIONS AND CHECKPOINT MANAGEMENT
# =============================================================================

def save_checkpoint_data(article_versions, preprocessor, output_path, checkpoint_name="steps_1_2"):
    """Save checkpoint data for review"""
    checkpoint_data = {
        'checkpoint_name': checkpoint_name,
        'timestamp': datetime.now().isoformat(),
        'article_metadata': article_versions.metadata,
        'processing_summary': preprocessor.get_processing_summary(),
        'validation_results': article_versions.metadata.get('validation_results', {}),
        'article_summary': article_versions.get_summary()
    }

    # Save to output directory
    checkpoint_file = f"{output_path}/{article_versions.article_name}_checkpoint_{checkpoint_name}.json"

    with open(checkpoint_file, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)

    print(f"\n💾 Checkpoint saved: {checkpoint_file}")
    return checkpoint_data

def run_steps_1_2(article_name, input_path, base_output_path):
    """Run steps 1-2 for a given article"""
    print(f"🚀 Starting Steps 1-2 for article: {article_name}")
    print(f"📂 Input path: {input_path}")

    # Setup output directories
    output_dir, archive_dir = setup_output_directories(base_output_path)

    # Step 1: Data Ingestion & Validation
    article_versions = ArticleVersions(article_name, input_path)
    article_versions.load_versions()
    validation_results = article_versions.validate_version_sequence()

    if not validation_results['is_valid']:
        print("❌ Cannot proceed: Missing required versions (draft and final)")
        return None, None

    # Step 2: Text Preprocessing
    preprocessor = TextPreprocessor()
    preprocessor.process_all_versions(article_versions)

    # Save checkpoint
    checkpoint_data = save_checkpoint_data(article_versions, preprocessor, output_dir)

    print(f"\n✅ Steps 1-2 completed successfully!")
    print(f"📊 Processing Summary:")
    for version, stats in preprocessor.get_processing_summary().items():
        print(f"  {version}: {stats['word_count']} words, {stats['sentence_count']} sentences")

    return article_versions, preprocessor

# Interactive input functions
def get_user_inputs():
    """Get user inputs for processing"""
    print("📝 Please provide the following information:")

    article_name = input("Enter article name (without .md extension): ").strip()
    input_path = input("Enter full path to input folder containing markdown files: ").strip()
    base_output_path = input("Enter full path to base output folder: ").strip()

    print(f"\n📋 Configuration:")
    print(f"  Article name: {article_name}")
    print(f"  Input path: {input_path}")
    print(f"  Output path: {base_output_path}")

    confirm = input("\nProceed with these settings? (y/n): ").strip().lower()

    if confirm == 'y':
        return article_name, input_path, base_output_path
    else:
        print("❌ Cancelled. Run get_user_inputs() again to restart.")
        return None, None, None

def process_article_interactive():
    """Process an article with interactive inputs"""
    article_name, input_path, base_output_path = get_user_inputs()

    if article_name and input_path and base_output_path:
        return run_steps_1_2(article_name, input_path, base_output_path)
    else:
        return None, None

print("📋 Ready to process your article!")
print("Run: article_versions, preprocessor = process_article_interactive()")
print("\nMake sure your markdown files are named:")
print("- draft-your-article-name.md")
print("- refined-your-article-name.md")
print("- edited-your-article-name.md")
print("- final-your-article-name.md")

📋 Example usage:
article_versions, preprocessor = run_steps_1_2(
    article_name='your-article-name',
    input_path='/content/drive/MyDrive/your-input-folder',
    base_output_path='/content/drive/MyDrive/your-output-folder'
)

Make sure your markdown files are named:
- draft-your-article-name.md
- refined-your-article-name.md
- edited-your-article-name.md
- final-your-article-name.md


In [ ]:
# =============================================================================
# CELL 5: SAMPLE DATA CREATOR (FOR TESTING ONLY)
# =============================================================================

def create_sample_files_for_testing(output_path):
    """Create sample markdown files for testing (run this once to test)"""
    sample_content = {
        'draft-': """# Sample Article

This is a draft article about artificial intelligence and its impact on society. AI has revolutionized many industries.

The technology continues to evolve rapidly. Machine learning algorithms are becoming more sophisticated every day.

We must consider the ethical implications of AI development.""",

        'refined-': """# Sample Article

This is a refined article examining artificial intelligence and its transformative impact on modern society. AI has fundamentally revolutionized numerous industries across the globe.

The technology continues to evolve at an unprecedented pace. Advanced machine learning algorithms are becoming increasingly sophisticated with each passing day.

We must carefully consider the complex ethical implications of AI development and deployment.""",

        'edited-': """# Sample Article

This comprehensive article examines artificial intelligence and its transformative impact on modern society. AI has fundamentally revolutionized numerous industries worldwide, reshaping how we work and live.

The technology continues to evolve at an unprecedented pace, driven by breakthrough innovations. Advanced machine learning algorithms are becoming increasingly sophisticated, enabling new applications we never thought possible.

We must carefully consider the complex ethical implications of AI development and deployment, ensuring responsible innovation for the benefit of humanity.""",

        'final-': """# Sample Article

This comprehensive article examines artificial intelligence and its transformative impact on modern society. AI has fundamentally revolutionized numerous industries worldwide, reshaping how we work, communicate, and live.

The technology continues to evolve at an unprecedented pace, driven by breakthrough innovations in computing power and algorithmic design. Advanced machine learning algorithms are becoming increasingly sophisticated, enabling new applications we never thought possible just a decade ago.

We must carefully consider the complex ethical implications of AI development and deployment, ensuring responsible innovation that serves the benefit of all humanity while mitigating potential risks."""
    }

    article_name = "sample-article"
    os.makedirs(output_path, exist_ok=True)

    for prefix, content in sample_content.items():
        filename = f"{prefix}{article_name}.md"
        filepath = os.path.join(output_path, filename)

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)

        print(f"Created: {filename}")

    return article_name, output_path

print("\n🧪 Sample data creator available for testing if needed.")
print("To create test files, run:")
print("sample_name, sample_path = create_sample_files_for_testing('/your/test/path')")


🧪 Sample data creator available for testing.
Uncomment and run the following to test with sample data:
sample_name, sample_path = create_sample_files_for_testing('/content/drive/MyDrive/test_input')
article_versions, preprocessor = run_steps_1_2(sample_name, sample_path, '/content/drive/MyDrive/test_output')
